In [1]:
import pandas as pd
import xlrd
import os
from collections import defaultdict

In [2]:
from PyPDF2 import PdfFileReader,PdfFileWriter

In [355]:
class Bill:
    def __init__(self, year, mon):
        self.year = year
        self.mon = mon
        self.sell_file = self.get_sell_file()
        content = xlrd.open_workbook(filename=self.sell_file,encoding_override='gbk')
        self.sell_df = pd.read_excel(content)
        self.commission_file = self.get_commission_file()
        content = xlrd.open_workbook(filename=self.commission_file,encoding_override='gbk')
        self.commission_df = pd.read_excel(content)
        #self.sell_df = pd.read_excel(self.sell_file,encoding='gbk')
    def gen_path(self, filename):
        return  './data/{0}_{1:02d}/{2}'.format(self.year, self.mon,filename)
    def get_sell_file(self):
        return self.gen_path('销售单据表.xls')
    def get_commission_file(self):
        return self.gen_path('提成明细表.xls')
    def get_total_income(self):
        #总收入
        self.total_income = 0
        #充值总收入
        self.total_recharge = 0
        #充次总收入
        self.total_time_recharge = 0
        for index, row in self.sell_df.iterrows():
            income_sf = row['实付']
            income_czk = row['储值卡']
            self.total_income += row['实付'] - row['储值卡']
            if row['单据类型'] == '购买次数':
                self.total_time_recharge += row['实付']
            if row['单据类型'] == '购买次数' or row['单据类型'] == '充值收费' or row['单据类型']=='开户收费':
                self.total_recharge += row['实付'] - row['储值卡']
        print(self.total_income)
        print(self.total_recharge)
        print(self.total_time_recharge)
    def get_true_income(self):
        # 实际项目总收入（会员+非会员）
        self.true_income = 0
        true_income_types = ['消费收银','快速消费']
        #会员实际项目总收入
        self.member_true_income = 0
        for index, row in self.sell_df.iterrows():
            if row['单据类型'] in true_income_types:
                if not pd.isnull(row['会员类型']):
                    self.member_true_income += row['实付']
                self.true_income += row['实付']
        print(self.member_true_income)
        print(self.true_income)
        
        #实际扣次项目
        self.true_time_dic = defaultdict(int)
        for index, row in self.commission_df.iterrows():
            if row['提成类型'] == '扣次消费':
                self.true_time_dic[row['名称']] += 1
        print(self.true_time_dic)
    def get_employee_commission(self):
        tmp_commission = self.commission_df.copy()
        selected_df = tmp_commission[['时间','员工','名称','支付','提成']]
        selected_df['时间'] = selected_df['时间'].apply(lambda x:'{0}-{1}'.format(x.month, x.day))
        for employee, df in selected_df.groupby(['员工']):
            print(employee)
            print(df)
            df = df[['时间','名称','支付','提成']]
            print(df['提成'].sum())
            df.to_csv(self.gen_path('{0}.csv'.format(employee)),index=False,encoding='gbk')
    def get_recharge_detail(self):
        tmp_df = self.sell_df.copy()
        recharge_df = tmp_df[tmp_df['单据类型'].isin(set(['购买次数','充值收费','开户收费']))]
        recharge_df = recharge_df[['单据类型','时间','实付','储值卡','会员姓名','会员卡号']]
        total_recharge = recharge_df['实付'].sum()-recharge_df['储值卡'].sum()
        des_sum = '累计充值：{0},各个会员充值情况如下\n'.format(total_recharge)
        des_sum += '==============================\n'
        for name, df in recharge_df.groupby(['会员卡号']):
            s = 0
            des = '{0}({1:d}):累计充值{2}元\n'.format(df.iloc[0]['会员姓名'],int(name),int(df['实付'].sum()-df['储值卡'].sum()))
            des_sum += des
            #print('{0}:{1}'.format(name,df['实付'].sum()-df['储值卡'].sum()))
            total_recharge += df['实付'].sum()-df['储值卡'].sum()
            s += df['实付'].sum()-df['储值卡'].sum()
            des = '明细如下:\n'
            df = df.sort_values(by=['单据类型'],ascending=False)

            for index, row in df.iterrows():
                if row['单据类型'] == '购买次数' and row['实付']>0:
                    des = des + '购买次数:' + str(int((row['实付']))) + ',时间:' + (str(row['时间'].month) + '-' + str(row['时间'].day))+'\n'
                    s -= row['实付']
                if row['单据类型'] == '充值收费' and s > 0:
                    des = des + '充值收费:' + str((int(row['实付']))) + ',时间:' + (str(row['时间'].month) + '-' + str(row['时间'].day))+'\n'
                    s -= row['实付']
                if row['单据类型'] == '开户收费' and s > 0:
                    des = des + '开户收费:' + str((int(row['实付']))) + ',时间:' + (str(row['时间'].month) + '-' + str(row['时间'].day))+'\n'
                    s -= row['实付']
            #print(des)
            des_sum += des
            des_sum += '==============================\n'
            f = open(self.gen_path('会员充值明细_{0}_{1}.txt'.format(self.year,self.mon)),'w')
            f.write(des_sum)
            f.close()
        return des_sum
    def get_nomember_detail(self):
        tmp_df = self.sell_df.copy()
        tmp_df['not_member'] = tmp_df['会员卡号'].apply(lambda x:pd.isnull(x))
        nomember_detail_df = tmp_df[tmp_df['not_member']]
        nomember_detail_df = nomember_detail_df[['单据类型','实付','时间']]
        #print(nomember_detail_df)
        des_not_member = ''
        des_not_member += '散客累计消费:{0}\n明细如下：\n'.format(nomember_detail_df['实付'].sum())
        for index, row in nomember_detail_df.iterrows():
            des_not_member += '{0}:'.format(row['单据类型']) + str((int(row['实付']))) + ',时间:' + (str(row['时间'].month) + '-' + str(row['时间'].day))+'\n'
        f = open(self.gen_path('散客消费明细_{0}_{1}.txt'.format(self.year,self.mon)),'w')
        f.write(des_not_member)
        f.close()
        return des_not_member

In [356]:
bill = Bill(2020, 1)

In [357]:
bill.get_sell_file()

'./data/2020_01/销售单据表.xls'

In [358]:
bill.get_true_income()

8149.5199999999995
17586.52
defaultdict(<class 'int'>, {'暖宫艾灸': 16, '精油开背': 3, '皮肤清洁护理': 3, '美睫': 1})


In [359]:
bill.sell_file

'./data/2020_01/销售单据表.xls'

In [360]:
bill.get_total_income()

30377.0
20940.0
6160.0


In [361]:
bill.commission_file

'./data/2020_01/提成明细表.xls'

In [362]:
bill.get_employee_commission()

孙妮
       时间  员工    名称       支付      提成
1    1-15  孙妮   NaN   496.00  143.20
3    1-22  孙妮   NaN  1000.00   50.00
4    1-22  孙妮   NaN   888.00  355.20
5    1-22  孙妮   NaN  1000.00   50.00
6    1-22  孙妮   NaN   500.00   25.00
8    1-22  孙妮   NaN   263.20   46.00
9    1-21  孙妮   NaN  1000.00   50.00
10   1-21  孙妮   NaN  1000.00   50.00
11   1-21  孙妮   NaN    28.00   11.20
12   1-21  孙妮   NaN  1000.00   50.00
13   1-21  孙妮   NaN   269.00  107.60
14   1-18  孙妮   NaN  1000.00   50.00
15   1-18  孙妮   NaN   252.00  100.80
16   1-18  孙妮   NaN   300.00   12.00
17   1-18  孙妮   NaN    62.00   24.80
21   1-15  孙妮   NaN  1000.00   25.00
23   1-15  孙妮   NaN   311.00  124.40
24   1-13  孙妮    美睫  1300.00   65.00
25   1-13  孙妮   NaN  1000.00   50.00
27   1-13  孙妮   NaN   306.50   93.20
29   1-12  孙妮   NaN  1000.00   25.00
31   1-12  孙妮   NaN   526.00  182.80
32   1-22  孙妮   NaN    60.00   24.00
34   1-22  孙妮   NaN    60.00   24.00
35   1-11  孙妮   NaN   500.00   50.00
36   1-11  孙妮   NaN    67.20   26.8

/Users/fanlinpeng/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [363]:
des_sum = bill.get_recharge_detail()

In [364]:
des_sum

'累计充值：20940.0,各个会员充值情况如下\n==============================\n黄婷(60006):累计充值500元\n明细如下:\n充值收费:500,时间:1-10\n==============================\n袁月(60015):累计充值1000元\n明细如下:\n购买次数:1000,时间:1-21\n==============================\n慧慧姐(60026):累计充值1000元\n明细如下:\n充值收费:1000,时间:1-9\n==============================\n褚娟(60027):累计充值1000元\n明细如下:\n充值收费:1000,时间:1-9\n==============================\n刘红贵(60028):累计充值2860元\n明细如下:\n购买次数:880,时间:1-10\n购买次数:1980,时间:1-10\n==============================\n徐毅琼(60029):累计充值1000元\n明细如下:\n充值收费:500,时间:1-11\n充值收费:500,时间:1-17\n==============================\n王女士(60030):累计充值500元\n明细如下:\n充值收费:500,时间:1-11\n==============================\n蔡佩君(60031):累计充值1000元\n明细如下:\n充值收费:1000,时间:1-12\n==============================\n陈芳(60032):累计充值1000元\n明细如下:\n充值收费:1000,时间:1-13\n==============================\n郭燕萍(60035):累计充值1300元\n明细如下:\n购买次数:1300,时间:1-13\n==============================\n筱蓓(60036):累计充值1000元\n明细如下:\n充值收费:1000,时间:1-15\n==============================\n婷婷(60037):累计充值1000元\n明细如下:\n购买次数:1000,

In [365]:
des_not_member = bill.get_nomember_detail()